In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
%matplotlib inline

import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# Case study - zbiór `Boston House Dataset`

## Model prostej regresji liniowej
1. Podziel dane na część treningową i na część testową w stosunku 7:3.


2. Interesuje nas zależność wielkości ceny domu od średniej liczby pokoi w domu (zmienne `PRICE` i `RM`). Określ zmienną zależną i niezależną, a następnie:

      a) Narysuj wykres rozproszenia tych zmiennych. 
      
      b) Czy zmienne te są zależne? 
      
      c) Oblicz dla nich współczynnik korelacji Pearsona. 


3. Wyznacz funkcję regresji wielkości ceny domu względem średniej liczby pokoi. Narysuj ją na wykresie rozproszenia.


4. Zweryfikuj dopasowanie modelu:

    a) Czy istnieje związek pomiędzy zmienną zależną a niezależną?
    
    b) Czy wyraz wolny jest istotny w modelu?
    
    c) Oblicz i zinterpretuj wartość współczynnika determinacji $R^2$.
    
    d) Sprawdź, czy spełnione jest założenie o rozkładzie normalnym reszt.
    
    e) Zweryfikuj dopasowanie modelu, sporządź wykresy diagnostyczne: wykres wartości przewidywanych względem prawdziwych, wykres residuów względem wartości dopasowanych.
    
    f) Wyznacz i porównaj jakość dopasowania i predykcji za pomocą różnych miar (np. MSE, MAE, $R^2$).

## Model wielorakiej regresji liniowej

5. Wyznacz model liniowy najlepiej opisujący zależność zależność wielkości ceny domu od pozostałych dostępnych zmiennych:

    a) Czy istnieje związek pomiędzy zmienną zależną a którąkolwiek ze zmiennych niezależnych?
    
    b) Dla których zmiennych niezależnych Xj, j=1,...,10 można odrzucić hipotezę zerową  H0: βj = 0?
    
    c) Biorąc pod uwagę odpowiedź na poprzednie pytanie, wyznacz mniejszy model w oparciu tylko o zmienne, które faktycznie mają istotny wpływ na wielkość sprzedaży.
    
    d) Porównaj dopasowanie obu modeli (większego i mniejszego), sporządź wykresy diagnostyczne.
    
    e) Wyznacz i porównaj jakość dopasowania i predykcji dla modelu mniejszego i większego.

## Regularyzacja

6. Zastosuj regularyzację

    a) grzbietową
    
    b) Lasso


## Model regresji wielomianowej

7. Dopasuj model regresji wielomianowej:
        
    a) Znajdź optymalny stopień wielomianu.
    
    b) Zweryfikuj dopasowanie modelu, sporządź wykresy diagnostyczne.
    
    c) Zastosuj regularyzację znajdując najlepszy parametr $\alpha$.

## Podsumowanie

8. Dokonaj porównania dopasowanych powyżej modeli regresji pod względem błędu dopasowania i błędu predykcji (dane końcowe przedstaw za pomocą tabeli). 
    
**Pamiętaj o graficznej ewaluacji modeli regresji!!!**

In [4]:
boston = pd.read_csv("boston.csv")
X, y = boston.iloc[:, boston.columns != 'MEDV'], boston['MEDV']
X


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48


In [ ]:
#https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html - opis kolumn

#Model prostej regresji liniowej

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

print(X_train.shape[0] / X.shape[0], X_test.shape[0] / X.shape[0])

0.6996047430830039 0.30039525691699603


# Model wielorakiej regresji liniowej

In [68]:
X_train_with_const = sm.add_constant(X_train, prepend=True, has_constant='skip')
X_test_with_const = sm.add_constant(X_test, prepend=True, has_constant='skip')

model_full = sm.OLS(endog=y_train, exog=X_train_with_const)
fitted_model_full = model_full.fit()
fitted_model_full.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.765
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     85.00
Date:                Sat, 14 Dec 2024   Prob (F-statistic):           2.44e-98
Time:                        16:47:12   Log-Likelihood:                -1034.2
No. Observations:                 354   AIC:                             2096.
Df Residuals:                     340   BIC:                             2151.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.9813      6.043      4.796      0.000      17.094      40.868
CRIM          -0.1010      0.035     -2.909      0.004      -0.169      -0.033
ZN             0.0400      0.016      2.551      0.011       0.009       0.071
INDUS          0.0754      0.072      1.045      0.297      -0.067       0.217
CHAS           0.2646      1.063      0.249      0.804      -1.826       2.355
NOX          -14.3409      4.595     -3.121      0.002     -23.380      -5.302
RM             4.8332      0.485      9.971      0.000       3.880       5.787
AGE           -0.0074      0.015     -0.488      0.626      -0.037       0.023
DIS           -1.3267      0.226     -5.879      0.000      -1.771      -0.883
RAD            0.2735      0.075      3.660      0.000       0.126       0.420
TAX           -0.0130      0.004     -3.059      0.002      -0.021      -0.005
PTRATIO       -0.9566      0.150     -6.372      0.000      -1.252      -0.661
B              0.0064      0.003      1.965      0.050   -7.31e-06       0.013
LSTAT         -0.4866      0.061     -8.032      0.000      -0.606      -0.367
==============================================================================
Omnibus:                      164.467   Durbin-Watson:                   2.149
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1012.790
Skew:                           1.865   Prob(JB):                    1.19e-220
Kurtosis:                      10.399   Cond. No.                     1.55e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.55e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Regularyzacja

In [6]:
# Regresja liniowa
model_linear_regression = LinearRegression()
model_linear_regression.fit(X_train, y_train)

LinearRegression()

In [10]:
# Regularyzacja grzbietowa alfa = 1 
model_ridge_regression = make_pipeline(
    StandardScaler(),
    Ridge(alpha=1)
)
model_ridge_regression.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=1))])

In [15]:
# Regularyzacja grzbietowa alfa = opt
estimator_ridge = make_pipeline(
    StandardScaler(),
    Ridge()
)

grid = GridSearchCV(
    estimator=estimator_ridge,
    param_grid = {'ridge__alpha': np.linspace(0.01, 15, 100)},
    scoring='neg_mean_squared_error',
    cv=10
)
grid.fit(X_train, y_train)
grid_best = grid.best_estimator_

In [22]:
# Regularyzacja lasso alpha = 1
model_lasso_regression = make_pipeline(
    StandardScaler(),
    Lasso(alpha=1)
)
model_lasso_regression.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=1))])

In [33]:
# Regularyzacja lasso alpha opt
estimator_lasso = make_pipeline(
    StandardScaler(),
    Lasso(max_iter = 10000)
)

grid_lasso = GridSearchCV(
    estimator=estimator_lasso,
    param_grid = {'lasso__alpha': np.arange(0.1, 10, 0.1)},
    scoring='neg_mean_squared_error',
    cv=10
)
grid_lasso.fit(X_train, y_train)
grid_best = grid_lasso.best_estimator_

In [27]:
def pred_model(model, X, y):
    mse = metrics.mean_squared_error(y_true=y, y_pred=model.predict(X))
    mae = metrics.median_absolute_error(y_true=y, y_pred=model.predict(X))

    results = {}
    results['mse'] = mse
    results['mae'] = mae
    return results

In [34]:
results_linear_regression = pred_model(model_linear_regression, X_test, y_test)
results_ridge_alpha_1 = pred_model(model_ridge_regression, X_test, y_test)
results_ridge_alpha_opt = pred_model(grid.best_estimator_, X_test, y_test)
results_lasso_alpha_1 = pred_model(model_lasso_regression, X_test, y_test)
results_lasso_alpha_opt = pred_model(grid_lasso.best_estimator_, X_test, y_test)

print(f'Linear regression\n{results_linear_regression}')
print(f'Regularyzacja grzbietowa alfa=1 \n{results_ridge_alpha_1}')
print(f'Regularyzacja grzbietowa alfa=opt \n{results_ridge_alpha_opt}')
print(f'Regularyzacja lasso alfa=1 \n{results_lasso_alpha_1}')
print(f'Regularyzacja lasso alfa=opt \n{results_lasso_alpha_opt}')


Linear regression
{'mse': 28.40585481050838, 'mae': np.float64(2.7094922194474087)}
Regularyzacja grzbietowa alfa=1 
{'mse': 28.422212815746555, 'mae': np.float64(2.7210653799367783)}
Regularyzacja grzbietowa alfa=opt 
{'mse': 28.597102513983057, 'mae': np.float64(2.5405577105040678)}
Regularyzacja lasso alfa=1 
{'mse': 35.51098770160136, 'mae': np.float64(2.9195904526917715)}
Regularyzacja lasso alfa=opt 
{'mse': 29.92149070202161, 'mae': np.float64(2.865511607807477)}


In [ ]:
# prowadzacego kod: 


In [36]:
def compute_metrics(model, X_test, y_test):
    preds = model.predict(X_test)
    mse = metrics.mean_squared_error(y_true=y_test, y_pred=preds)
    mae = metrics.median_absolute_error(y_true=y_test, y_pred=preds)
    return {'mse': mse, 'mae': mae}

In [38]:
# Regresja liniowa
m1_linear = LinearRegression().fit(X_train, y_train)
compute_metrics(m1_linear, X_test, y_test)

{'mse': 28.40585481050838, 'mae': np.float64(2.7094922194474087)}

In [40]:
# Regresja grzbietowa
m2_ridge = make_pipeline(StandardScaler(), Ridge())
m2_ridge.fit(X_train,y_train)
ridge_gridCV = GridSearchCV(
    estimator=m2_ridge,
    param_grid = {'ridge__alpha': np.arange(0.1, 10, 0.1)},
    scoring='neg_mean_squared_error',
    cv=10
)
ridge_gridCV.fit(X_train, y_train)
compute_metrics(ridge_gridCV.best_estimator_, X_test, y_test)

{'mse': 28.598327749354475, 'mae': np.float64(2.540632876566354)}

In [41]:
# Regularyzacja lasso
m3_lasso = make_pipeline(StandardScaler(), Lasso(max_iter=10000))
m3_lasso.fit(X_train,y_train)
lasso_gridCV = GridSearchCV(
    estimator=m3_lasso,
    param_grid = {'lasso__alpha': np.arange(0.1, 10, 0.1)},
    scoring='neg_mean_squared_error',
    cv=10
)
lasso_gridCV.fit(X_train, y_train)
compute_metrics(lasso_gridCV.best_estimator_, X_test, y_test)

{'mse': 29.92149070202161, 'mae': np.float64(2.865511607807477)}

In [42]:
compute_metrics(m1_linear, X_test, y_test)
compute_metrics(ridge_gridCV.best_estimator_, X_test, y_test)
compute_metrics(lasso_gridCV.best_estimator_, X_test, y_test)

{'mse': 29.92149070202161, 'mae': np.float64(2.865511607807477)}

## Regresja wielomianowa

In [ ]:
# regresja wielo + reg
# potegowa do max 5 potegi

In [44]:
poly_ridge_estimator = make_pipeline(
    PolynomialFeatures(include_bias = False),
    StandardScaler(),
    Ridge()
)
poly_ridge_estimator

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(include_bias=False)),
                ('standardscaler', StandardScaler()), ('ridge', Ridge())])

In [55]:
grid_ridge_poly = GridSearchCV(
    estimator=poly_ridge_estimator,
    param_grid={
        'polynomialfeatures__degree': [1, 2, 3, 4, 5],
        'ridge__alpha': np.arange(0.1, 10, 0.5)
    },
    scoring='neg_mean_squared_error',
    cv=10
)

grid_ridge_poly.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures(include_bias=False)),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'polynomialfeatures__degree': [1, 2, 3, 4, 5],
                         'ridge__alpha': array([0.1, 0.6, 1.1, 1.6, 2.1, 2.6, 3.1, 3.6, 4.1, 4.6, 5.1, 5.6, 6.1,
       6.6, 7.1, 7.6, 8.1, 8.6, 9.1, 9.6])},
             scoring='neg_mean_squared_error')

In [56]:
results_poly_ridge_alpha_opt = pred_model(grid_ridge_poly.best_estimator_, X_test, y_test)
print(results_poly_ridge_alpha_opt)

{'mse': 16.499453385244465, 'mae': np.float64(1.7256673932025688)}


In [57]:
grid_ridge_poly.best_params_ 

{'polynomialfeatures__degree': 2, 'ridge__alpha': np.float64(1.6)}

In [58]:
poly_lasso_estimator = make_pipeline(
    PolynomialFeatures(include_bias = False),
    StandardScaler(),
    Lasso(max_iter=10000)
)
poly_lasso_estimator


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(include_bias=False)),
                ('standardscaler', StandardScaler()),
                ('lasso', Lasso(max_iter=10000))])

In [64]:
grid_lasso_poly = GridSearchCV(
    estimator=poly_lasso_estimator,
    param_grid={
        'polynomialfeatures__degree': [1, 2, 3,4,5],
        'lasso__alpha': np.arange(0.01, 1, 0.05)
    },
    scoring='neg_mean_squared_error',
    cv=10
)

grid_lasso_poly.fit(X_train, y_train)
results_poly_lasso_alpha_opt = pred_model(grid_lasso_poly.best_estimator_, X_test, y_test)
print(results_poly_lasso_alpha_opt)

{'mse': 17.693245250145576, 'mae': np.float64(1.7757595726497257)}


In [65]:
grid_lasso_poly.best_params_ 

{'lasso__alpha': np.float64(0.01), 'polynomialfeatures__degree': 2}

In [66]:
grid_lasso_poly.best_estimator_

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(include_bias=False)),
                ('standardscaler', StandardScaler()),
                ('lasso', Lasso(alpha=np.float64(0.01), max_iter=10000))])

### Wyniki


In [67]:
print("Regresja liniowa")
print(f'Linear regression\n{results_linear_regression}')
print(f'Regularyzacja grzbietowa alfa=1 \n{results_ridge_alpha_1}')
print(f'Regularyzacja grzbietowa alfa=opt \n{results_ridge_alpha_opt}')
print(f'Regularyzacja lasso alfa=1 \n{results_lasso_alpha_1}')
print(f'Regularyzacja lasso alfa=opt \n{results_lasso_alpha_opt}')
print("Regresja wielomianowa")
print(f'Regularyzacja grzbietowa alfa=opt \n{results_poly_ridge_alpha_opt}')
print(f'Regularyzacja lasso alfa=opt \n{results_poly_lasso_alpha_opt}')


Regresja liniowa
Linear regression
{'mse': 28.40585481050838, 'mae': np.float64(2.7094922194474087)}
Regularyzacja grzbietowa alfa=1 
{'mse': 28.422212815746555, 'mae': np.float64(2.7210653799367783)}
Regularyzacja grzbietowa alfa=opt 
{'mse': 28.597102513983057, 'mae': np.float64(2.5405577105040678)}
Regularyzacja lasso alfa=1 
{'mse': 35.51098770160136, 'mae': np.float64(2.9195904526917715)}
Regularyzacja lasso alfa=opt 
{'mse': 29.92149070202161, 'mae': np.float64(2.865511607807477)}
Regresja wielomianowa
Regularyzacja grzbietowa alfa=opt 
{'mse': 16.499453385244465, 'mae': np.float64(1.7256673932025688)}
Regularyzacja lasso alfa=opt 
{'mse': 17.693245250145576, 'mae': np.float64(1.7757595726497257)}


In [ ]:
# mniejsza alfa mniejsza regularyzacja
# jak wymuszam duze alfa to moze sie zdarzyc ze mamy wiekszy stopein wielomianu 